<center><h1>Accidents Corporels de la Route en France</h1></center>
<br>
<center><i>Projet Data Visualization - IGR204</i></center>
<center><i>Télécom ParisTech</i></center>

## General imports

In [1]:
# General
import numpy as np
import pandas as pd
import itertools
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Graph
import matplotlib.pyplot as plt
import matplotlib.offsetbox as offsetbox
import seaborn as sns
import plotly
import plotly.plotly as py
plotly.tools.set_credentials_file(username='anatoli.db', api_key='xWsOWcl2vbDBL2BK4yYL')

# I - Import preprocessed data 

In [2]:
# Import datframe
df = pd.read_csv('preprocessed_data.csv')

# Show head
df.head()

,Unnamed: 0,Num_Acc,num_veh,vehicule,grav,secu,road,pieton,sexe
0,0,200500000001,A01,Voiture,Blessure,Oui,Départementale,Non,M
1,1,200500000001,B02,Voiture,Blessure,Oui,Départementale,Non,F
2,2,200500000001,B02,Voiture,Indemne,Oui,Départementale,Non,M
3,3,200500000001,B02,Voiture,Indemne,Oui,Départementale,Non,M
4,4,200500000001,B02,Voiture,Indemne,Oui,Départementale,Non,M


## IV  - Sankey Diagram

In [3]:
# Dict source
source_vehicule = {'Voiture':[0, 'rgb(253,168,90)', 'rgba(253,168,90, 1)', 'rgba(253,168,90, 0.5)', 'rgba(253,168,90, 0.1)'],
                   'Moto':[1, 'rgb(119,59,117)', 'rgba(119,59,117, 1)', 'rgba(119,59,117, 0.5)', 'rgba(119,59,117, 0.1)'],
                   'Vélo':[2, 'rgb(14,167,181)', 'rgba(14,167,181, 1)', 'rgba(14,167,181, 0.5)', 'rgba(14,167,181, 0.1)'],
                   'Camion':[3, 'rgb(12,69,125)', 'rgba(12,69,125, 1)', 'rgba(12,69,125, 0.5)', 'rgba(12,69,125, 0.1)'],
                   }


source_injury = {'Décès':[4,'rgb(66,78,91)', 'rgba(66,78,91, 1)', 'rgba(66,78,91, 0.3)'],
                 'Blessure':[5, 'rgb(209, 123, 166)', 'rgba(209, 123, 166, 1)',  'rgba(209, 123, 166, 0.3)'],
                 'Indemne':[6, 'rgb(122,163,94)', 'rgba(122,163,94, 1)', 'rgba(122,163,94, 0.3)' ]}

source_security = {'Non':[7, 'rgb(235,178,89)', 'Pas de sécurité'], 
                   'Oui':[8, 'rgb(138,207,188)', 'Sécurité']}

source_obstacle = {'Oui':[9, 'rgb(128, 20, 7)', 'rgba(128, 20, 7, 0.4)', 'Piéton']}


In [4]:
# Build Sankey Source and target and node
node = []
source = []
target = []
value = []
color = []
link_color = []
link_label = []


for key_v in source_vehicule:

    for i, key_i in enumerate(source_injury):
         
        # Link source 
        source.append(source_vehicule[key_v][0])

        # Link target
        target.append(source_injury[key_i][0])

        # Link value 
        value.append(df[(df['vehicule'] == key_v) & (df['grav'] == key_i)].shape[0])

        # Link color 
        link_color.append(source_vehicule[key_v][2 + i])

        # Link label 
        link_label.append('Vehicule: {}<br>Gravité: {}'.format(key_v, key_i))
        
        
for i, key_i in enumerate(source_injury):

    # Security link
    for j, key_s in enumerate(source_security):

        # Link source 
        source.append(source_injury[key_i][0])

        # Link target
        target.append(source_security[key_s][0])

        # Link value 
        value.append(df[(df['secu'] == key_s) & (df['grav'] == key_i)].shape[0])

        # Link color 
        link_color.append(source_injury[key_i][j + 2])

        # Link label 
        link_label.append('Gravité: {}<br>Utilisation de la sécurité: {}'.format(key_i, key_s))
        
    # Obstacle link
    if key_i in ['Décès', 'Blessure']:
        for k, key_o in enumerate(source_obstacle):

            # Link source 
            source.append(source_injury[key_i][0])

            # Link target
            target.append(source_obstacle[key_o][0])

            # Link value 
            value.append(df[(df['secu'] == key_s) & (df['grav'] == key_i) & (df['pieton'] == key_o)].shape[0])

            # Link color 
            link_color.append(source_obstacle[key_o][2])

            # Link label 
            link_label.append('Gravité: {}<br>Utilisation de la sécurité: {}<br>Piéton: {}'.format(key_i, key_s, key_o))



# Create node Label and colors for vehicule
for key_v in source_vehicule:
    node.append(key_v)
    color.append(source_vehicule[key_v][1])
    
# Create node label and colors for injury
for key_i in source_injury:
    node.append(key_i)
    color.append(source_injury[key_i][1])
    
# Create node Label and colors for security
for key_s in source_security:
    node.append(source_security[key_s][-1])
    color.append(source_security[key_s][1])
    
# Create node Label and colors for pedestrian
for key_o in source_obstacle:
    node.append(source_obstacle[key_o][-1])
    color.append(source_obstacle[key_o][1])


In [5]:
# Build datframe to feed sankey diagram
sankey = pd.DataFrame(columns=['Source','Target','Value','Link_Color', 'Link_Label'])
sankey['Source'] = source
sankey['Target'] = target
sankey['Value'] = value
sankey['Link_Color'] = link_color
sankey['Link_Label'] = link_label

sankey.to_csv('complete_dataset.csv',';')


In [9]:
data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    
    node = dict(
      pad = 40,
      thickness = 40,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label =  node,
      color = color
    ),
    
    link = dict(
      source = sankey['Source'].dropna(axis=0, how='any'),
      target = sankey['Target'].dropna(axis=0, how='any'),
      value = sankey['Value'].dropna(axis=0, how='any'),
      color = sankey['Link_Color'].dropna(axis=0, how='any'),
      label = sankey['Link_Label'].dropna(axis=0, how='any')
  )
)

layout =  dict(
    title = "Accidents de la route en France entre 2006 et 2018<br><i>Sankey schéma</i>",
    height = 800,
    width = 1000,
    font = dict(
      size = 15
    ),    
)

fig = dict(data=[data_trace], layout=layout)
py.iplot(fig, validate=False)

/anaconda3/envs/msbgd/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

